This notebook generates the umap embeddings of the images in the dataset. The embeddings are saved in a parquet file.

In [1]:
import argparse
import pathlib

import numpy as np
import pandas as pd
import umap

In [2]:
# # set the arg parser
# parser = argparse.ArgumentParser(description="UMAP on a matrix")

# parser.add_argument("--data_mode", type=str, default="CP", help="data mode to use")

# # get the args
# args = parser.parse_args()

# # set data mode to either "CP" or "scDINO" or "combined" or "terminal"
# data_mode = args.data_mode

data_mode = "terminal"

In [ ]:
# set the paths to the data
CP_fs_sc_profiles_path = pathlib.Path(
    "../../data/CP_feature_select/profiles/features_selected_profile.parquet"
).resolve(strict=True)
combined_profiles_path = pathlib.Path(
    "../../data/CP_scDINO_features/combined_CP_scDINO_norm_fs.parquet"
).resolve(strict=True)
scDINO_fs_profiles_path = pathlib.Path(
    "../../data/scDINO/CLS_features_annotated_normalized_feature_selected.parquet"
).resolve(strict=True)

CP_endpoint_profiles_path = pathlib.Path(
    "../../data/CP_feature_select/endpoints/features_selected_profile.parquet"
).resolve(strict=True)

output_path = pathlib.Path("../../data/umap/").resolve()
output_path.mkdir(parents=True, exist_ok=True)

In [4]:
if data_mode == "CP":
    # read the data
    profiles_df = pd.read_parquet(CP_fs_sc_profiles_path)
elif data_mode == "combined":
    # read the data
    profiles_df = pd.read_parquet(combined_profiles_path)
elif data_mode == "scDINO":
    # read the data
    profiles_df = pd.read_parquet(scDINO_fs_profiles_path)
elif data_mode == "terminal":
    # read the data
    profiles_df = pd.read_parquet(CP_endpoint_profiles_path)
else:
    raise ValueError("data_mode must be either 'CP' or 'scDINO' or 'combined'")
print(profiles_df.shape)
# show all columns
pd.set_option("display.max_columns", None)
profiles_df.head()

(9918, 543)


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_ImageNumber_1,Metadata_ImageNumber_2,Metadata_ImageNumber_3,Metadata_Nuclei_Number_Object_Number,Metadata_Image_FileName_AnnexinV,Metadata_Image_FileName_DNA,Metadata_Image_PathName_AnnexinV,Metadata_Image_PathName_DNA,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_coordinates_x,Metadata_track_id,Metadata_t,Metadata_y,Metadata_x,Metadata_id,Metadata_parent_track_id,Metadata_parent_id,Metadata_coordinates_y,Metadata_distance,Cytoplasm_AreaShape_Area,Metadata_Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MedianRadius,Cytoplasm_AreaShape_MinFeretDiameter,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Perimeter,Cytoplasm_AreaShape_Solidity,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,Cytoplasm_AreaShape_Zernike_3_3,Cytoplasm_AreaShape_Zernike_5_1,Cytoplasm_AreaShape_Zernike_6_0,Cytoplasm_AreaShape_Zernike_7_1,Cytoplasm_AreaShape_Zernike_7_3,Cytoplasm_AreaShape_Zernike_7_7,Cytoplasm_AreaShape_Zernike_9_1,Cytoplasm_AreaShape_Zernike_9_5,Cytoplasm_AreaShape_Zernike_9_9,Cytoplasm_Correlation_Correlation_AnnexinV_DNA,Cytoplasm_Correlation_K_AnnexinV_DNA,Cytoplasm_Correlation_K_DNA_AnnexinV,Cytoplasm_Correlation_Manders_AnnexinV_DNA,Cytoplasm_Correlation_Manders_DNA_AnnexinV,Cytoplasm_Correlation_Overlap_AnnexinV_DNA,Cytoplasm_Granularity_1_AnnexinV,Cytoplasm_Granularity_1_DNA,Cytoplasm_Intensity_IntegratedIntensityEdge_DNA,Cytoplasm_Intensity_IntegratedIntensity_AnnexinV,Cytoplasm_Intensity_IntegratedIntensity_DNA,Cytoplasm_Intensity_MADIntensity_AnnexinV,Cytoplasm_Intensity_MassDisplacement_AnnexinV,Cytoplasm_Intensity_MassDisplacement_DNA,Cytoplasm_Intensity_MaxIntensity_AnnexinV,Cytoplasm_Intensity_MaxIntensity_DNA,Cytoplasm_Intensity_MeanIntensityEdge_DNA,Cytoplasm_Intensity_MedianIntensity_AnnexinV,Cytoplasm_Intensity_StdIntensity_AnnexinV,Cytoplasm_Intensity_StdIntensity_DNA,Cytoplasm_Intensity_UpperQuartileIntensity_AnnexinV,Cytoplasm_RadialDistribution_FracAtD_AnnexinV_4of4,Cytoplasm_RadialDistribution_FracAtD_DNA_1of4,Cytoplasm_RadialDistribution_FracAtD_DNA_2of4,Cytoplasm_RadialDistribution_FracAtD_DNA_3of4,Cytoplasm_RadialDistribution_MeanFrac_DNA_1of4,Cytoplasm_RadialDistribution_RadialCV_AnnexinV_1of4,Cytoplasm_RadialDistribution_RadialCV_AnnexinV_2of4,Cytoplasm_RadialDistribution_RadialCV_AnnexinV_3of4,Cytoplasm_RadialDistribution_RadialCV_DNA_1of4,Cytoplasm_RadialDistribution_RadialCV_DNA_2of4,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_1_1,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_2_0,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_2_2,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_3_1,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_3_3,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_4_0,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_4_2,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_5_1,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_5_3,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_5_5,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_6_0,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_6_2,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_7_1,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_7_3,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_7_5,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_7_7,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_8_0,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_8_2,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_8_4,Cytoplasm_Radia

In [5]:
# filter the data and drop nan values
print(profiles_df.shape)
# drop nan values in non metadata columns
profiles_df = profiles_df.dropna(
    subset=profiles_df.columns[~profiles_df.columns.str.contains("Meta")]
)
print(profiles_df.shape)

(9918, 543)
(9364, 543)


In [6]:
# get the metadata columns
metadata_cols = profiles_df.columns.str.contains("Metadata_")
metadata_df = profiles_df.loc[:, metadata_cols]
features_df = profiles_df.loc[:, ~metadata_cols]

# set the umap parameters
umap = umap.UMAP(
    n_neighbors=15,
    n_components=2,
    metric="euclidean",
    random_state=42,
    min_dist=0.1,
    n_epochs=500,
    learning_rate=1,
)

# fit the umap model
umap.fit(features_df)

# transform the data
umap_transformed = umap.transform(features_df)

# create a dataframe with the transformed data
umap_df = pd.DataFrame(
    umap_transformed, columns=["UMAP0", "UMAP1"], index=features_df.index
)

# combine the metadata and umap dataframes
umap_df = pd.concat([metadata_df, umap_df], axis=1)
print(umap_df.shape)
umap_df.head()

/home/lippincm/miniforge3/envs/timelapse_env/lib/python3.11/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(9364, 35)


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_ImageNumber_1,Metadata_ImageNumber_2,Metadata_ImageNumber_3,Metadata_Nuclei_Number_Object_Number,Metadata_Image_FileName_AnnexinV,Metadata_Image_FileName_DNA,Metadata_Image_PathName_AnnexinV,Metadata_Image_PathName_DNA,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_coordinates_x,Metadata_track_id,Metadata_t,Metadata_y,Metadata_x,Metadata_id,Metadata_parent_track_id,Metadata_parent_id,Metadata_coordinates_y,Metadata_distance,Metadata_Cytoplasm_AreaShape_BoundingBoxArea,UMAP0,UMAP1
0,1,C-09,153,Staurosporine,39.06,positive,1,0002,13.0,1.0,1,1,1,1.0,1.0,1.0,C-09_F0002_T0014_Z0001_C05_illumcorrect_aligne...,C-09_F0002_T0014_Z0001_C01_illumcorrect_aligne...,/home/lippincm/4TB_A/live_cell_timelapse_apopt...,/home/lippincm/4TB_A/live_cell_timelapse_apopt...,22.111519,282.453419,"[22.111519045433685, 282.4534189995411]",18,13.0,282.0,22.0,14000022.0,-1,13000021.0,"[22.0, 282.0]",0.466932,-1.615781,12.346190,1.673221
1,1,C-09,153,Staurosporine,39.06,positive,1,0002,13.0,3.0,3,7,1,1.0,1.0,7.0,C-09_F0002_T0014_Z0001_C05_illumcorrect_aligne...,C-09_F0002_T0014_Z0001_C01_illumcorrect_aligne...,/home/lippincm/4TB_A/live_cell_timelapse_apopt...,/home/lippincm/4TB_A/live_cell_timelapse_apopt...,62.282197,459.101326,"[62.28219696969697, 459.10132575757575]",32,13.0,459.0,62.0,14000036.0,-1,13000037.0,"[62.0, 459.0]",0.299837,-1.235406,7.819421,-3.434050
2,1,C-09,153,Staurosporine,39.06,positive,1,0002,13.0,6.0,6,6,1,1.0,1.0,6.0,C-09_F0002_T0014_Z0001_C05_illumcorrect_aligne...,C-09_F0002_T0014_Z0001_C01_illumcorrect_aligne...,/home/lippincm/4TB_A/live_cell_timelapse_apopt...,/home/lippincm/4TB_A/live_cell_timelapse_apopt...,61.548811,798.433795,"[61.548810791622294, 798.4337948171814]",52,13.0,798.0,62.0,14000063.0,-1,13000066.0,"[62.0, 798.0]",0.625899,-0.631163,13.778338,-1.745288
3,1,C-09,153,Staurosporine,39.06,positive,1,0002,13.0,8.0,8,2,1,1.0,1.0,2.0,C-09_F0002_T0014_Z0001_C05_illumcorrect_aligne...,C-09_F0002_T0014_Z0001_C01_illumcorrect_aligne...,/home/lippincm/4TB_A/live_cell_timelapse_apopt...,/home/lippincm/4TB_A/live_cell_timelapse_apopt...,24.291609,1225.714351,"[24.291609353507564, 1225.714351215039]",89,13.0,1226.0,24.0,14000105.0,-1,13000098.0,"[24.0, 1226.0]",0.408205,-1.345526,8.513001,-2.869495
4,1,C-09,153,Staurosporine,39.06,positive,1,0002,13.0,9.0,9,3,1,1.0,1.0,3.0,C-09_F0002_T0014_Z0001_C05_illumcorrect_aligne...,C-09_F0002_T0014_Z0001_C01_illumcorrect_aligne...,/home/lippincm/4TB_A/live_cell_timelapse_apopt...,/home/lippincm/4TB_A/live_cell_timelapse_apopt...,5.220736,1312.013378,"[5.2207357859531776, 1312.0133779264213]",162,13.0,1312.0,5.0,14000112.0,-1,13000108.0,"[5.0, 1312.0]",0.221141,-1.376585,7.507778,-3.905384


In [7]:
# save the umap dataframe
umap_df.to_parquet(f"../../data/umap/{data_mode}_umap_transformed.parquet")